# Laboratorio 5 Parte 2

### Reducción de dimensión por extracción de características con PCA y LDA

### Universidad de Antioquia

### Facultad de Ingeniería

### Ingeniería de Sistemas

### Ude@ - 2018-I

#### Profesor: Antonio Tamayo Herrera

## Estudiantes

Nombre: Rodrigo Antonio Porras Martinez

Céduda: 1039459720

Nombre: David Yepes Isaza

Cédula: 1020455542

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de extracción de características (PCA y LDA) y usar SVM para resolver un problema de clasificación multietiqueta o multiclase.

Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography

#### Abstract: 
The dataset consists of measurements of fetal heart rate (FHR) and uterine contraction (UC) features on cardiotocograms classified by expert obstetricians.
	

#### Data Set Information:

2126 fetal cardiotocograms (CTGs) were automatically processed and the respective diagnostic features measured. The CTGs were also classified by three expert obstetricians and a consensus classification label assigned to each of them. Classification was both with respect to a morphologic pattern (A, B, C. ...) and to a fetal state (N, S, P). Therefore the dataset can be used either for 10-class or 3-class experiments.

#### Attribute Information:

LB - FHR baseline (beats per minute)

AC - # of accelerations per second

FM - # of fetal movements per second

UC - # of uterine contractions per second

DL - # of light decelerations per second

DS - # of severe decelerations per second

DP - # of prolongued decelerations per second

ASTV - percentage of time with abnormal short term variability

MSTV - mean value of short term variability

ALTV - percentage of time with abnormal long term variability

MLTV - mean value of long term variability

Width - width of FHR histogram

Min - minimum of FHR histogram

Max - Maximum of FHR histogram

Nmax - # of histogram peaks

Nzeros - # of histogram zeros

Mode - histogram mode

Mean - histogram mean

Median - histogram median

Variance - histogram variance

Tendency - histogram tendency

CLASS - FHR pattern class code (1 to 10)

NSP - fetal state class code (N=normal (1); S=suspect (2); P=pathologic (3))


Analice la base de datos, sus características, su variable de salida y el contexto del problema.

Antes de iniciar a ejecutar las celdas, debe instalar la librería mlxtend que usaremos para los laboratorios de reducción de dimensión.
Para hacerlo solo tiene que usar el siguiente comando: sudo pip install mlxtend. También puede consultar la guía oficial de instalación
    de esta librería: https://rasbt.github.io/mlxtend/installation/

Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [3]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA
from mlxtend.feature_extraction import LinearDiscriminantAnalysis as LDA

import time

#cargamos la bd de entrenamiento
db = np.loadtxt('DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

#Para darle formato de entero a la variable de salida

Y_l = []
for i in Y:
    Y_l.append(int(i))
Y = np.asarray(Y_l)

print ("Dimensiones de la base de datos de entrenamiento. dim de X: " + str(np.shape(X)) + "\tdim de Y: " + str(np.shape(Y)))


Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)


En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [4]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

#Para calcular el costo computacional
tiempo_i = time.time()

#Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

#Implemetamos la metodología de validación

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    #Aquí se entran y se valida el modelo sin hacer selección de características
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1

print("\nError de validación sin aplicar extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

#print str(ypred)
#print str(y_test) 


Error de validación sin aplicar extracción: 0.07712817787226504 +/- 0.05442325724156325


Tiempo total de ejecución: 0.9090361595153809 segundos.


## Ejercicio 1

1.1 Al aplicar PCA es necesario estandarizar los datos? Si, No y por qué? En qué consiste dicha estandarización?

R/: En PCA si es neceario estandarizar los datos, consiste en centrar los datos, es decir, para centrar los datos es necesario extraerle la media a las caracteristicas y que la media sea cero. Se debe hacer porque en el tratamiento matricial que sea hace posteriormente necesitan que los datos esten centrados. 
    
1.2 La proyección de los datos que realiza PCA se hace buscando minimizar o maximizar algo? Explique.

R/: La proyección de los datos que realiza PCA se hace buscando minimizar las distancias que hay entre el punto $X_i$ y su proyección sobre la dirección del vector unitario $u_1$. De donde se tiene que la distancia es $|x_i-z_j u_1|^2$ y para minimizarla hay que encontrar el vector $u_1$ que maximice $z^2_j$

## Ejercicio 2

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de extracción de características. https://rasbt.github.io/mlxtend/user_guide/feature_extraction/

In [8]:
#Feature Extraction Function
#Recibe 2 parámetros: 1. el tipo de método de extracción (pca o lda como string), 2. el número componentes (para pca)
#o el número de discriminantes (para lda)

#Para este laboratorio solo se le pedirá trabajar con PCA, LDA es opcional.

def extract_features(tipo, n):
    
    if tipo == 'pca':
    
        ext = PCA(n_components=n)
    
        return ext

    elif tipo == 'lda':
        
        ext = LDA(n_discriminants=n)
        
        return ext
    
    else:
        print ("Ingrese un método válido (pca o lda)\n")

#Para calcular el costo computacional
tiempo_i = time.time()

#Estandarizamos los datos
X = standardize(X)

#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    #Aquí se aplica la extracción de características por PCA
    #Complete el código
    #Complete el código llamando el método extract_features. Tenga en cuenta lo que le pide el ejercicio 3.1
    ex = extract_features('pca',21) 

    #Fit de PCA
    #Complete el código con el fit correspondiente
    ex = ex.fit(X)
    
    #Transforme las variables y genere el nuevo espacio de características de menor dimensión
    #complete el código aquí para hacer la transformación
    X_ex = ex.transform(X)
    
    
    #Aquí se aplica la extracción de características por LDA
    
    #OPCIONAL
    '''
    ex = #Complete el código llamando el método extract_features. Tenga en cuenta lo que le pide el ejercicio 3.1

    #Fit de LDA
    ex = #Complete el código con el fit correspondiente
    
    #Transforme las variables y genere el nuevo espacio de características de menor dimensión
    X_ex = #complete el código aquí para hacer la transformación
    '''
    
    #Se aplica CV-10
    
    X_train, X_test = X_ex[train_index], X_ex[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
   
    #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1
        

print("\nError de validación aplicando extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print("\nEficiencia en validación aplicando extracción: " + str((1-np.mean(Errores))*100) + "%" )

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

#print str(ypred)
#print str(y_test)


Error de validación aplicando extracción: 0.07430463282841704 +/- 0.04391645244688941

Eficiencia en validación aplicando extracción: 92.5695367171583%


Tiempo total de ejecución: 0.9602289199829102 segundos.


## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de componentes principales a tener en cuenta (use 2, 10, 19 y 21 componentes principales) para PCA y complete la siguiente tabla de resultados:

<table>
  <tr>
    <th>Técnica</th>
    <th># de componentes principales</th>
    <th>Error de validación</th>
    <th>IC (std)</th>
    <th>Tiempo de ejecución</th>
  </tr>
  <tr>
    <td>SVM sin extracción</td>
    <td>N/A</td>
    <td>0.07712817787226504 +/- 0.05442325724156325</td>
    <td>N/A</td>
    <td>0.9090361595153809</td>
  </tr>
  <tr>
    <td>SVM + PCA</td>
    <td>2</td>
    <td>0.22142793870139074 +/- 0.17001475908401714</td>
    <td>77.85720612986093%</td>
    <td>1.8265130519866943 s</td>
  </tr>
  <tr>
    <td>SVM + PCA</td>
    <td>10</td>
    <td>0.09169324120825582 +/- 0.061564467997647386</td>
    <td>90.83067587917442%</td>
    <td>1.1364219188690186</td>
  </tr>
  <tr>
    <td>SVM + PCA</td>
    <td>19</td>
    <td>0.07193949862698203 +/- 0.04841386304281984</td>
    <td>92.8060501373018%</td>
    <td>1.1248812675476074</td>
  </tr>
  <tr>
    <td>SVM + PCA</td>
    <td>21</td>
    <td>0.07430463282841704 +/- 0.04391645244688941</td>
    <td>92.5695367171583%</td>
    <td>0.9602289199829102</td>
  </tr>
      
</table>


3.2 Analizando los resultados del punto anterior que puede decir de la viabilidad de aplicar PCA para hacer reducción de dimensión en este problema?

R/: Tiene buena eficiencia y el costo computacional es muy poco. Con 10 componentes logramos obtener una eficiencia del 90.8% reduciendo la dimencicion a la mitad en tan solo 1.13 segundos. Hay una mejor eficiciencia con con el numero 19 de componentes, es del 92.8% pero no es muy eficiente al reducir la dimesion, tenemos 19 componentes principales(nuevas caracteristicas o proyecciones) y las caracteristicas que se tenian eran 22.


3.3 Explique en sus palabras la principal ventaja que tiene LDA sobre PCA para resolver problemas de clasificación.

R/: PCA busca las direcciones de proyecciones que son eficientes para manejar los datos, el LDA busca direcciones que son eficientes para discriminación, es decir que permiten una mejor separación de las clases en el espacio de menor dimensión.

3.3 Explique en sus palabras las diferencias que existen entre los métodos de selección de características y los métodos de extracción de características vistos en el curso.

R/: Los metodos de seleccion de caracteristicas se encargan de elegir las caracteristicas mas aportantes al problema probando con diferentes tecnicas, cuanto afecta cada caracteristica o conjutno de caractaristicas a la solucion del problema, mientras que los metodos de extraccion se valen de tranformaciones lineales en busqueda de datos que realmente aporten a la solucion del problema.
Cabe notar que mientras en la seleccion de caracterisitcas el conjunto resultado es interpetable ya que es un subconjunto del conjunto principal en la extraccion las caracterisitcas resultados no son interpetables.